In [1]:
import pandas as pd
import numpy as np

In [2]:
# データの集約について見ていく
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})
df

,data1,data2,key1,key2
0,-0.394873,0.136662,a,one
1,2.716306,0.079991,a,two
2,-0.818959,-1.149582,b,one
3,0.353429,0.238134,b,two
4,0.691158,0.120254,a,one


In [3]:
# 分位点はquantile関数
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    2.311276
b    0.236190
Name: data1, dtype: float64

In [4]:
# 最大値と最小値の差を返す関数を作る
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.111179,0.056671
b,1.172388,1.387715


In [5]:
# describeでpandasにビルトインされた統計量を計算する関数を全て実行できる
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.004197  1.579036 -0.394873  0.148143  0.691158  1.703732   
b      2.0 -0.232765  0.829003 -0.818959 -0.525862 -0.232765  0.060332   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     2.716306   3.0  0.112302  0.029160  0.079991  0.100122  0.120254   
b     0.353429   2.0 -0.455724  0.981263 -1.149582 -0.802653 -0.455724   

                          
           75%       max  
key1                      
a     0.128458  0.136662  
b    -0.108795  0.238134

In [6]:
tips = pd.read_csv('../plot/tips.csv')
# 合計金額に対するチップの割合を示すカラム
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [7]:
# ビルトインされている統計量計算関数を文字列ベースで実行できる
grouped = tips.groupby(['sex', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [8]:
# 複数の関数を実行できる
# describeと違い自前実装の関数も実行できる
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [9]:
# (名前, 関数)という形で引数に渡せば任意の関数を実行できてかつ列名を指定できる
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [10]:
# 複数の任意の列に対して任意の複数の関数を実行する
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [11]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [12]:
# 複数の任意の列に対して任意の複数の名前付き関数を実行する
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [13]:
# タプルでなくディクショナリでも名前付き関数を実行できる
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [14]:
# 任意の列に対して複数の関数を実行は、ディクショナリでもできる
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

In [15]:
# カラムのインデックス付けないでgroupbyする
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771
